In [16]:
import pandas as pd
import numpy as np
import re

In [14]:
train_data = pd.read_csv('train.csv', usecols=['tweet','label'])

In [69]:
def clean_text(text):
    reg = text.lower()
    reg = re.sub(r"(.)\1+", r" \1 ", reg)
    reg =  re.sub(r'\!', ' VOSKZNAK ', reg)
    reg =  re.sub(r'\.', ' POINT ', reg)
    reg =  re.sub(r'\?', ' QUESTION ', reg)
    reg = re.sub('\d+', ' NUMBER ', reg)
    reg = re.sub('\n', ' ', reg)
    reg = re.sub('\)', ' HAPPYSMILE ', reg)
    reg = re.sub('\(', ' DADSMILE ', reg)
    reg = re.sub('[-@#$%^&*(){}|_:;.,/?~<>]', '', reg)
    reg = re.sub(" +", " ", reg)


    return reg

In [70]:
clean_text(
    'душкрипцшип  з9843уцшо----кипзшоципе    цоуип)))) лйцотв )))  !!! ! @ # $ $KJNLKJB *&&'
)

'душкрипцшип з NUMBER уцшо кипзшоципе цоуип HAPPYSMILE лйцотв HAPPYSMILE VOSKZNAK VOSKZNAK kjnlkjb '

In [63]:
train_data.tweet.apply(clean_text)

0         user when a father is dysfunctional and is so...
1        user user thanks for lyft credit i can't use c...
2                                      bihday your majesty
3        model i love u take with u a l the time in urð...
4                        factsguide society now motivation
                               ...                        
31957    ate user isz that yo u QUESTION ðððð...
31958     to s e nina turner on the airwaves trying to ...
31959    listening to sad songs on a monday morning otw...
31960    user sikh temple vandalised in in calgary wso ...
31961                      thank you user for you fo l ow 
Name: tweet, Length: 31962, dtype: object

In [64]:
train_data

,label,tweet
0,0,@user when a father is dysfunctional and is s...
1,0,@user @user thanks for #lyft credit i can't us...
2,0,bihday your majesty
3,0,#model i love u take with u all the time in ...
4,0,factsguide: society now #motivation
...,...,...
31957,0,ate @user isz that youuu?ðððððð...
31958,0,to see nina turner on the airwaves trying to...
31959,0,listening to sad songs on a monday morning otw...
31960,1,"@user #sikh #temple vandalised in in #calgary,..."


In [65]:
clean_text(
    'душкрипцшип  зуцшо----кипзшоципе    цоуип)))) лйцотв )))  !!! ! @ # $ $KJNLKJB *&&'
)

'душкрипцшип зуцшо кипзшоципе цоуип HAPPYSMILE лйцотв HAPPYSMILE VOSKZNAK VOSKZNAK kjnlkjb '

In [19]:
list(train_data.tweet.iloc[:100])

[' @user when a father is dysfunctional and is so selfish he drags his kids into his dysfunction.   #run',
 "@user @user thanks for #lyft credit i can't use cause they don't offer wheelchair vans in pdx.    #disapointed #getthanked",
 '  bihday your majesty',
 '#model   i love u take with u all the time in urð\x9f\x93±!!! ð\x9f\x98\x99ð\x9f\x98\x8eð\x9f\x91\x84ð\x9f\x91\x85ð\x9f\x92¦ð\x9f\x92¦ð\x9f\x92¦  ',
 ' factsguide: society now    #motivation',
 '[2/2] huge fan fare and big talking before they leave. chaos and pay disputes when they get there. #allshowandnogo  ',
 ' @user camping tomorrow @user @user @user @user @user @user @user dannyâ\x80¦',
 "the next school year is the year for exams.ð\x9f\x98¯ can't think about that ð\x9f\x98\xad #school #exams   #hate #imagine #actorslife #revolutionschool #girl",
 'we won!!! love the land!!! #allin #cavs #champions #cleveland #clevelandcavaliers  â\x80¦ ',
 " @user @user welcome here !  i'm   it's so #gr8 ! ",
 ' â\x86\x9d #ireland consume